# MSTR - Robotics: Schema Monitor

### Start Notebook

In [1]:
from mstr_robotics.read_out_prj_obj import read_schema,read_gen

from mstr_robotics.report import cube,rep
from mstr_robotics.mstr_pandas import df_helper
from mstr_robotics.mstr_classes import  get_conn

import pandas as pd
import json

i_rep=rep()
i_cube=cube()
i_read_gen=read_gen()
i_df_helper=df_helper()
i_read_schema=read_schema()
run_prop_d={}


#in my environment the PA project is on the same maschine
#open parameter files
with open('D:\\shared_drive\\Python\\mstr_robotics\\mstr_robotics\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)

#set user credentials and open a connection to the i-server
username = user_d["username"]
password =user_d["password"]
base_url= user_d["base_url"]
pa_base_url= user_d["base_url"]

project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn=get_conn(base_url=base_url,username=username,password=password)
conn.select_project(project_id)
           
pa_conn=get_conn(base_url=pa_base_url,username=username,password=password)

Connection to MicroStrategy Intelligence Server has been established.
No project selected.
Connection to MicroStrategy Intelligence Server has been established.
No project selected.


### Set id's for underlying cubes

In [2]:
#Schema information
cube_folder_id="9981CB214C79D75EC342A781539A4E5A" #Schema Monitor

cbe_tbl_att_fct_mapping_name="tbl_att_fct_mapping"
cbe_tbl_att_fct_mapping_id="3673EC9146EE4DF3517725BBA97A4B36"
cbe_tbl_att_fct_mapping_id=None


cbe_att_form_exp_name="att_form_exp"
cbe_att_form_exp_id="3E312E64489040609C0A63800A4E3BAE"

cbe_fact_exp_name="fact_exp"
cbe_fact_exp_id="5D8A265D4A08641AAB76DAB74440A41C"

cbe_table_exp_name="table_exp"
cbe_table_exp_id="745D4D304697491769F4F4B41E906BE0"

# blended object and PA/EM information
cbe_obj_depn_rep_col_usage_name="obj_depn_rep_col_usage"
cbe_obj_depn_rep_col_usage_id=None



### read out schema table mappings

In [3]:
#read out the schema & builds a cube upload file for attributes, facts & table definitions

schema_mappings_d=i_read_schema.table_mappings(conn=conn,run_prop_d=run_prop_d)

#better readabillity in the next cell
att_form_exp_df=schema_mappings_d["att_form_exp_df"]
fact_exp_df=schema_mappings_d["fact_exp_df"]
table_df=schema_mappings_d["table_df"]

#prepare data for cubing
att_form_exp_d_l=[{"df":att_form_exp_df,"tbl_name":"att_form_exp_df", "update_policy":"Replace"}]
fact_exp_d_l=[{"df":fact_exp_df,"tbl_name":"fact_exp_df", "update_policy":"Replace"}]
table_d_l=[{"df":table_df,"tbl_name":"table_df", "update_policy":"Replace"}]

In [4]:
table_df.columns

Index(['project_id', 'table_id', 'table_name', 'physicalTable_id',
       'physicalTable_name', 'column_id', 'column_name', 'column_versionId',
       'column_dateModified', 'column_dateCreated', 'column_subType',
       'column_dataType', 'column_precision', 'column_scale'],
      dtype='object')

In [5]:
#sometimes it's easier to join the datasets outside of a dossier
#here we build a primary key data set by joinning dataframes
tbl_att_fct_df = pd.merge(table_df[["project_id","table_id","table_name", "column_id","column_name","physicalTable_id","physicalTable_name"]],
                          att_form_exp_df[["project_id", 'table_id', "column_id", "attribute_id","attribute_name", "form_expressionId","form_name"]],
                          on=['project_id', 'table_id', 'column_id'], how='left')
i_df_helper.clean_double_col(df=tbl_att_fct_df)
tbl_att_fct_df = pd.merge(tbl_att_fct_df, 
                          fact_exp_df[["project_id", "column_id","fact_id","fact_name","fact_expressionId"]],
                          on=['project_id', 'column_id'], how='left')
i_df_helper.clean_double_col(df=tbl_att_fct_df)
tbl_att_fct_df["phys_tbl_col_id"]=tbl_att_fct_df["physicalTable_id"]+ "_"+ tbl_att_fct_df["column_id"]
tbl_att_fct_d_l=[{"df":tbl_att_fct_df,"tbl_name":"tbl_att_fct_df", "update_policy":"Replace"}]

## report usage

### Platform Analytics

In [6]:
#some customers are using PA other not
#I only pass over the project as prompt answer here
pa_project_id="40DA7A1549651FD60A9D39AAB7EC77B0"
pa_project_element_id="7224027087873511424" # here you see the element_id of MicroStrategy Tutorial in PA
pa_conn.select_project(pa_project_id)
prompt_answ=f'{{"prompts": [{{"id": "EF66D49D40D0E799E1D1909A65085E97", "type": "ELEMENTS","answers":[{{"id": "h{pa_project_element_id};3A0BE6C741DE7CDD4A4C01925B5A04E9"}}]}} ]}}'

print("Element Prompt Answer= " + prompt_answ)

Element Prompt Answer= {"prompts": [{"id": "EF66D49D40D0E799E1D1909A65085E97", "type": "ELEMENTS","answers":[{"id": "h7224027087873511424;3A0BE6C741DE7CDD4A4C01925B5A04E9"}]} ]}


In [7]:
#report stats
report_usage="BF03DD1443947C6192D0229B4A3D7EF8"

#get data
instance_id=i_rep.open_Instance(conn=pa_conn,report_id=report_usage)
prp_resp=i_rep.set_inst_prompt_ans(conn=pa_conn,report_id=report_usage,instance_id=instance_id, prompt_answ=prompt_answ)
report_usage_df=i_rep.rep_to_dataframe(conn=pa_conn,report_id=report_usage,instance_id=instance_id)

#clean data to get the same column names as EM
adjust_col_names_to_em = {"project_id@GUID":"project_id", "report_id@GUID":"report_id", "year_id@ID":"year_id"
                          , "month_id@ID":"month_id"}
report_usage_df.rename(columns=adjust_col_names_to_em, inplace=True)

#prepare cube upload
rep_usage_d_l=[{"df":report_usage_df,"tbl_name":"rep_usage_df", "update_policy":"Replace"}]

In [8]:
# db views & column stats
views_cols_usage="9A96A6474DB13EA45D4B96B4485463C8"
#get data
instance_id=i_rep.open_Instance(conn=pa_conn,report_id=views_cols_usage)
prp_resp=i_rep.set_inst_prompt_ans(conn=pa_conn,report_id=report_usage,instance_id=instance_id, prompt_answ=prompt_answ)
view_col_usage_df=i_rep.rep_to_dataframe(conn=pa_conn,report_id=views_cols_usage,instance_id=instance_id)
#clean data to get the same column names as EM
adjust_col_names_to_em = {"project_id@GUID":"project_id", "report_id@GUID":"report_id", "year_id@ID":"year_id"
                          , "month_id@ID":"month_id", "db_table@GUID":"physicalTable_id", 
                          "column@Name":"column_name","column@GUID":"column_id"}
view_col_usage_df.rename(columns=adjust_col_names_to_em, inplace=True)
view_col_usage_df.reset_index()
#prepare cube upload
view_col_usage_d_l=[{"df":view_col_usage_df,"tbl_name":"view_col_usage_df", "update_policy":"Replace"}]

## Object Information 

### REST API search

In [9]:
#advantage is very flexible & includes security
#disadvantage: takes ages!!!!!

#count_only_fg=True counts dependent objects
#count_only_fg=False list dependent objects

#info_level = "base" provides the folder id
#info_level = "base_path" provides the full folder path 

obj_type_l=["3"] #only check reports
obj_depn_df =i_read_gen.chk_by_obj_type(conn=conn #,project_id=project_id
                          ,obj_type_l=obj_type_l, info_level = "base_path",mtdi_id=None
                          ,count_only_fg=True,run_prop_d=run_prop_d)

#adjusting the name here, simplyfies the mapping of the attributes in the dossier
obj_depn_df.rename(columns={'id': 'report_id'}, inplace=True)

#prepare cube upload
obj_depn_d_l=[{"df":obj_depn_df,"tbl_name":"obj_depn_df", "update_policy":"Replace"}]

### Flag objects

In [10]:
#using this cell we can flag objects up on a certain parent folderId

flag_path_d_l=[{"in_duty":"Daniel",
                "flag_folder_id":"032A5E114A59D28267BDD8B6D9E58B22"},
              {"in_duty":"Verena",
                "flag_folder_id": "F025A94B4C03B6DCEE0F5D9DA825DA67"},
              {"in_duty":"Dolores",
                "flag_folder_id":"92ADD0F84D07AC532AD03BA0F92A836B"},
              {"in_duty":"Ed",
                "flag_folder_id": "7F16A4B811E58ED317D50080EFF554EA"},
                {"in_duty":"Mike",
                "flag_folder_id": "D64C532E4E7FBA74D29A7CA3576F39CF"}]

#apply the flags
depn_obj_df_flag=i_df_helper.flag_folder_df(flag_path_d_l=flag_path_d_l,obj_df=obj_depn_df,path="path_ids"
                                            ,flag_folder_id="flag_folder_id", merge_key_l=['project_id', 'report_id'])
#prepare cube upload
obj_depn_d_l=[{"df":depn_obj_df_flag,"tbl_name":"obj_depn_df", "update_policy":"Replace"}]

## Cubing

### load cubes

In [11]:
# if mtdi_id = None a new cube is created
# if mtdi_id <> None an existing cube is update
# if the mtdi_id does not exists or you provide different data structures
# you'll get an error msg


#primery key cube schema usage
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=tbl_att_fct_d_l,mtdi_id=cbe_tbl_att_fct_mapping_id , cube_name=cbe_tbl_att_fct_mapping_name,folder_id=cube_folder_id,force=True)
#attribute form expressions 
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=att_form_exp_d_l,mtdi_id=cbe_att_form_exp_id , cube_name=cbe_att_form_exp_name,folder_id=cube_folder_id,force=True)
#fact expressions
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=fact_exp_d_l,mtdi_id=cbe_fact_exp_id , cube_name=cbe_fact_exp_name,folder_id=cube_folder_id,force=True)
#table definitions
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=table_d_l,mtdi_id=cbe_table_exp_id , cube_name=cbe_table_exp_name,folder_id=cube_folder_id,force=True)

#define multi table cube
#ensure that derived attributes are mapped 
#over common column names
obj_depn_rep_col_usage_l=[]
obj_depn_rep_col_usage_l.extend(obj_depn_d_l)
obj_depn_rep_col_usage_l.extend(rep_usage_d_l)
obj_depn_rep_col_usage_l.extend(view_col_usage_d_l)
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_depn_rep_col_usage_l, mtdi_id=cbe_obj_depn_rep_col_usage_id,cube_name=cbe_obj_depn_rep_col_usage_name ,folder_id=cube_folder_id,force=True)
conn.close()
pa_conn.close()

Created super cube 'tbl_att_fct_mapping' with ID: '3673EC9146EE4DF3517725BBA97A4B36'.


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00,  3.14it/s, rows=2200]


Super cube 'tbl_att_fct_mapping' published successfully.
SuperCube object named: 'att_form_exp' with ID: '3E312E64489040609C0A63800A4E3BAE'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00, 13.71it/s, rows=495]


Super cube 'att_form_exp' published successfully.
SuperCube object named: 'fact_exp' with ID: '5D8A265D4A08641AAB76DAB74440A41C'


...

Super cube 'fact_exp' published successfully.
SuperCube object named: 'table_exp' with ID: '745D4D304697491769F4F4B41E906BE0'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00,  3.81it/s, rows=646]


Super cube 'table_exp' published successfully.
Created super cube 'obj_depn_rep_col_usage' with ID: '44FED4824123C1B96B070C80D5BBBD8C'.


Uploading 3/3: 100%|██████████| 1/1 [00:00<00:00, 14.93it/s, rows=1609]


Super cube 'obj_depn_rep_col_usage' published successfully.
Connection to MicroStrategy Intelligence Server has been closed.
Connection to MicroStrategy Intelligence Server has been closed.


### Tips & tricks
Pls. find a link to a PDF we're I place advices when they came up. Pls. keep in mind, that most of us, are doing this the first time...

https://github.com/magerdaniel/MSTR_Robotics/blob/main/mstr_files_and_ppt/MSTR%20Robotics%20Schema%20Monitor%20Tips.pdf